In [1]:
from docx import Document
from docx.enum.style import WD_STYLE_TYPE
import sys
import pandas as pd

import re
from zhon.hanzi import punctuation
import string
import numpy as np
from docx.shared import RGBColor
from copy import deepcopy

from docx.text.paragraph import Paragraph
from docx.oxml.xmlchemy import OxmlElement

import Levenshtein

# Load Fund Firm Pair

In [2]:
excel_name = "/Users/yangguangyao/Desktop/qianqian/jijindanwei.xlsx"
xls = pd.ExcelFile(excel_name)

df_fund = pd.read_excel(xls, 'fund', header=None)
df_fund.columns = ["cn", "en"]

df_firm = pd.read_excel(xls, 'firm', header=None)
df_firm.columns = ["cn", "en"]

# Get Firm From Input

In [3]:
def process_firm_string(s):
    # remove number 
    s1 = ''.join([i for i in s if not i.isdigit()])
    # remove chinese puncs
    s2 = re.sub(u"[{}]+".format(punctuation + string.punctuation), " ", s1) # 需要将str转换为unicode
    # remove
    s3 = filter(lambda x: len(x)>3, s2.split())
    s4 = " ".join(s3)
    return s4

In [35]:
doc_path = "/Users/yangguangyao/Desktop/qianqian/t1-firm-A.docx"
document = Document(doc_path)

orig_cn_firm = []
cn_firm = []
for p in document.paragraphs:
    if p.text and p.text[:2] == "AA":
        
        t1 = p.text[2:]
        t2 = process_firm_string(p.text[2:])
        
        orig_cn_firm.append(t1)
        cn_firm.append(t2)
        
        print t1
        print t2

（北京中医药大学，北京100029）
北京中医药大学
（1湖南中医药大学附属中西医结合医院针灸推拿康复科，长沙 4100061；2湖南中医药大学针灸推拿学院，长沙 410208）
湖南中医药大学附属中西医结合医院针灸推拿康复科 湖南中医药大学针灸推拿学院
（1中国中医科学院中药研究所，北京 100700；2河北大学附属医院中西医结合科，
中国中医科学院中药研究所 河北大学附属医院中西医结合科
（安徽中医药大学第一附属医院，合肥 230031）
安徽中医药大学第一附属医院
（1中日友好医院，北京 100029；2北京中医医院顺义医院，北京 101300）
中日友好医院 北京中医医院顺义医院
（1福建中医药大学中医学院，福州 350122；2福建中医药大学附属福州中医院，福州350001）
福建中医药大学中医学院 福建中医药大学附属福州中医院
（天津市中医药研究院附属医院肾病科，天津 300120）
天津市中医药研究院附属医院肾病科
（河南中医药大学，郑州 450005）
河南中医药大学
 [ 1北京市延庆区医院（北京大学第三医院延庆医院），北京 102100；2北京中医药大学
北京市延庆区医院 北京大学第三医院延庆医院 北京中医药大学


In [40]:
print cn_firm[3]

安徽中医药大学第一附属医院


# Find Corresponding En

In [6]:
def get_df_quiry(df_firm, firm_quiry):


    cn_list = []
    en_list = []
    dist_list = []

    for i in df_firm.iterrows():
        cn_name = i[1].cn
        en_name = i[1].en
        if type(cn_name) == type(u""):
            curr_dist = Levenshtein.distance(firm_quiry, cn_name)

            dist_list.append(curr_dist)
            cn_list.append(cn_name)
            en_list.append(en_name)

    df_quiry = pd.DataFrame({
            "cn": cn_list,
            "en": en_list,
            "d": dist_list
        })

    df_quiry2 = df_quiry.sort_values(by=['d'])[["cn", "en"]].head(5)
    
    return df_quiry2

In [121]:
print firm_quiry
df_quiry2

安徽中医药大学第一附属医院


,cn,en
152,广西中医药大学第一附属医院,The First Affiliated Hospital of Guangxi Unive...
165,广州中医药大学第一附属医院,No.1 Affiliated Hospital of Guangzhou Universi...
594,天津中医药大学第一附属医院,First Teaching Hospital of Tianjin University ...
284,湖南中医药大学第一附属医院,The First Hospital of Hunan University of Chin...
163,广州中医药大学第三附属医院,"No.3 Affiliated Hospital, Guangzhou University..."


# Write to World

In [147]:
document = Document()

n_row, n_col = df_quiry2.shape

# add firm_quiry
paragraph = document.add_paragraph().add_run(firm_quiry)
font = paragraph.font
font.color.rgb = RGBColor(0x42, 0x24, 0xE9)


# add firm_rst
table = document.add_table(rows=0, cols=n_col, style='Table Grid')
for i in df_quiry2.iterrows():
    cells = table.add_row().cells
    cells[0].text = i[1].cn
    cells[1].text = i[1].en

In [148]:
document.save("/Users/yangguangyao/Desktop/qianqian/t1.docx")

# All together

## only firm table

In [42]:
doc_path = "/Users/yangguangyao/Desktop/qianqian/0226.docx"
document = Document(doc_path)

rst_document = Document()



# part1
# find cn, en

cn_s_list = []
en_s_list = []
for ii, p in enumerate(document.paragraphs):
    
    # find firm tag
    if p.text and p.text[:3] == "AA1":
        cn_s_list.append(p)
        
    if p.text and p.text[:3] == "AA2":
        en_s_list.append(p)
        
# part2 parse
assert len(cn_s_list) == len(en_s_list)

cnt = 104
for i in range(len(cn_s_list)):
    
    p = cn_s_list[i]
    p_en = en_s_list[i]

    
    s_process = process_firm_string(p.text.strip()[3:])

    rst_document.add_paragraph(str(cnt), "Heading 1")

    rst_document.add_paragraph(p.text[3:])
    
    rst_document.add_paragraph(p_en.text[3:])

    # get seperate firm_quiry
    for firm_quiry in s_process.split():

        df_quiry2 = get_df_quiry(df_firm, firm_quiry)

        # write to the word
        n_row, n_col = df_quiry2.shape

        # add firm_quiry
        paragraph = rst_document.add_paragraph().add_run(firm_quiry)
        font = paragraph.font
        font.color.rgb = RGBColor(0x42, 0x24, 0xE9)

        # add firm_rst
        table = rst_document.add_table(rows=0, cols=n_col, style='Table Grid')
        for i in df_quiry2.iterrows():
            cells = table.add_row().cells
            cells[0].text = i[1].cn
            cells[1].text = i[1].en
            
    cnt +=1
            
rst_document.save("/Users/yangguangyao/Desktop/qianqian/firm_table.docx")  

## only fund table

In [36]:
doc_path = "/Users/yangguangyao/Desktop/qianqian/0226.docx"
document = Document(doc_path)

rst_document = Document()



# part1
# find cn, en

cn_s_list = []
en_s_list = []
for ii, p in enumerate(document.paragraphs):
    
    # find firm tag
    if p.text and p.text[:3] == "BB1":
        cn_s_list.append(p)
        
    if p.text and p.text[:3] == "BB2":
        en_s_list.append(p)
        
# part2 parse
assert len(cn_s_list) == len(en_s_list)

cnt = 104
for i in range(len(cn_s_list)):
    
    p = cn_s_list[i]
    p_en = en_s_list[i]

    s_process = process_firm_string(p.text[3:])

    rst_document.add_paragraph(str(cnt), "Heading 1")

    rst_document.add_paragraph(p.text[3:])
    
    rst_document.add_paragraph(p_en.text[3:])

    # get seperate firm_quiry
    for firm_quiry in s_process.split():

        if firm_quiry == u"基金资助":
            continue
            
        df_quiry2 = get_df_quiry(df_fund, firm_quiry)

        # write to the word
        n_row, n_col = df_quiry2.shape

        # add firm_quiry
        paragraph = rst_document.add_paragraph().add_run(firm_quiry)
        font = paragraph.font
        font.color.rgb = RGBColor(0x42, 0x24, 0xE9)

        # add firm_rst
        table = rst_document.add_table(rows=0, cols=n_col, style='Table Grid')
        for i in df_quiry2.iterrows():
            cells = table.add_row().cells
            cells[0].text = i[1].cn
            cells[1].text = i[1].en
            
    cnt +=1
            
rst_document.save("/Users/yangguangyao/Desktop/qianqian/fund_table.docx")  

## embed

In [188]:
def insert_paragraph_after(paragraph, text=None, style=None):
    """Insert a new paragraph after the given paragraph."""
    new_p = OxmlElement("w:p")
    paragraph._p.addnext(new_p)
    new_para = Paragraph(new_p, paragraph._parent)
    if text:
        new_para.add_run(text)
    if style is not None:
        new_para.style = style
    return new_para

In [242]:
doc_path = "/Users/yangguangyao/Desktop/qianqian/t1-firm-A.docx"
document = Document(doc_path)

rst_document = Document()

for p in document.paragraphs:

    # find firm tag
    if p.text and p.text[:2] == "AA":
        
        paragraph = insert_paragraph_after(p, "")
        s_process = process_firm_string(p.text[2:])
        
        # get seperate firm_quiry
        for firm_quiry in s_process.split():
            
            df_quiry2 = get_df_quiry(df_firm, firm_quiry)
            
            # write to the word
            n_row, n_col = df_quiry2.shape
            
            # add table
            table = rst_document.add_table(rows=0, cols=n_col, style='Colorful Grid')
            for i in df_quiry2.iterrows():
                cells = table.add_row().cells
                cells[0].text = i[1].cn
                cells[1].text = i[1].en
            tbl2, p2 = table._tbl, p._p
            new_tbl = deepcopy(tbl2)
            p2.addnext(new_tbl)
            
            # add firm_quiry
            paragraph = insert_paragraph_after(p)
            run = paragraph.add_run(firm_quiry)
            run.font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
            run.font.bold = True

            # add line split
            paragraph = insert_paragraph_after(p, "")
            paragraph = insert_paragraph_after(p, "")

            
#             # add firm_quiry
#             paragraph = document.add_paragraph().add_run(firm_quiry)
#             font = paragraph.font
#             font.color.rgb = RGBColor(0x42, 0x24, 0xE9)
            
#             # add firm_rst
#             #table = document.add_table(rows=0, cols=n_col, style='Table Grid')
#             table = document.add_table(rows=0, cols=n_col)
#             #table = document.add_table(rows=0, cols=n_col, style='Table Normal')
#             for i in df_quiry2.iterrows():
#                 cells = table.add_row().cells
#                 cells[0].text = i[1].cn
#                 cells[1].text = i[1].en
        paragraph = insert_paragraph_after(p, "")
                
document.save("/Users/yangguangyao/Desktop/qianqian/doc_embed.docx")  

# Get styles

In [20]:
doc_path = "/Users/yangguangyao/Desktop/qianqian/2019年02期69-77吴芊 - 已审.docx"
document = Document(doc_path)

In [21]:
style_title = document.styles["qq_title"]  # 题目：四号，加粗，居中
style_author = document.styles["qq_author"] # 作者，作者单位：五号，居中
style_kw = document.styles["qq_kw"] # 关键词，资金资助，摘要：五号，两端对齐

# Parse Paragraph

In [28]:
def parse_document(doc_path):
    """
    get sections
    """
    document = Document(doc_path)
    
    sections = []
    curr_section = []
    for p in document.paragraphs:
        if len(p.text) > 0:
            curr_section.append(p)
        else:
            if len(curr_section) > 0:
                sections.append(curr_section)
                curr_section = []
    if len(curr_section) > 0:
        sections.append(curr_section)
    return sections

In [29]:
doc_path = "/Users/yangguangyao/Desktop/qianqian/test.docx"
sections = parse_document(doc_path)

In [ ]:
def fix_section(section):
    # 题目，作者，单位，摘要，关键词
    assert len(sections[0]) == 5

In [13]:
document.paragraphs[4].style = style_title

In [15]:
document.save("t1.docx")

In [6]:
style_title = document.styles.add_style("qq_title", WD_STYLE_TYPE.PARAGRAPH)

ValueError: document already contains style 'qq_title'

In [42]:
style_title

NameError: name 'style_title' is not defined

In [39]:
style

_ParagraphStyle('qq_title') id: 4376747280

In [34]:
doc.styles.

<bound method Styles.default of <docx.styles.styles.Styles object at 0x104e158d0>>

# parse word

In [17]:
doc_path = "/Users/yangguangyao/Desktop/qianqian/2019年02期69-77吴芊 - 已审.docx"
document = Document(doc_path)

p = document.paragraphs[3]

In [18]:
print p.text
print p.style

三部《名医类案》中历代医家误用小柴胡汤医案探析
_ParagraphStyle('Normal') id: 4377429584


# Save

In [10]:
doc = Document()
doc.add_paragraph(u'对中国能源问题的思考','Title')

p = doc.add_paragraph(u'能源是经济社会发展的重要制约因素')
p.style = 'Heading 2'
doc.save('demo.docx')

In [5]:
doc_path = "/Users/yangguangyao/Desktop/qianqian/2019年02期69-77吴芊 - 已审.docx"

document = Document(doc_path)
for p in document.paragraphs:
    print p.text

#     print '===='
#     for r in p.runs:
#         print len(r.text)



69.研究报告
三部《名医类案》中历代医家误用小柴胡汤医案探析
王瑞婷,邹锦,张小勇,宋慧荣,罗亚敏,赵占领,吴印亮,陶晓华*
(北京中医药大学,北京100029)
摘要:小柴胡汤误治案是指在错误的病机下运用小柴胡汤,不惟旧病不除,反致祸患频出｡研究文献中的小柴胡汤误治医案有助于今人警惕小柴胡汤适应证中的疑似病症,本课题拟通过举例分析三部《名医类案》中的12例小柴胡汤误治医案,根据辨证施治过程中的关键环节,分类总结其失误原因,以期为临证运用小柴胡汤提供借鉴,经验包括:①发热性疾病,除首辨外感与内伤外,外感病人尚需辨别其属阴证或阳证,内伤病人需判断其属实证或虚证;②当少阳与阳明症状相兼时,需全面运用四诊合参以减少失误;③ “但见一症”思路必须在把握少阳枢机不利这一病机的前提下进行应用;④施治阶段,医者需耐心交代服药宜忌,以防治疗事倍功半｡
关键词:名医类案;小柴胡汤;误治

Analysis of medical cases which use Xiaochaihu Decoction in the wrong way in three series of Mingyi Leian
WANG Rui-ting, ZOU Jin, ZHANG Xiao-yong, SONG Hui-rong, LUO Ya-min, ZHAO Zhan-ling, WU Yin-liang, TAO Xiao-hua*
(Beijing University of Chinese Medicine, Beijing 100029, China)
Abstract: The mistreatment of Xiaochaihu Decoction refers to the use of Xiaochaihu Decoction under the wrong pathogenesis. In this case, not only does the old disease not be removed, but the disaster often occurs. The mistreatment of Xiaochaihu Decoction in the research literature helps people to be alert to the suspected 